In [ ]:
import pandas as pd
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
# from plotly.subplots import make_subplots
import xarray as xr
import datetime
import os,ast
# import plotly.graph_objects as go
# from sklearn import linear_model
# import statsmodels.api as sm
import glob
import warnings
# import ee
# import geopandas as gpd
import pickle
# from scipy.optimize import curve_fit
# from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings("ignore") 
import itertools
from joblib import Parallel, delayed
from multiprocessing import Pool
import time

import dask.dataframe as dd
from dask.multiprocessing import get
import ast
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
regions = ['NE','NW','SE','SW']

def convert_string_to_list(str_tuple):
    if pd.notna(str_tuple):
        tuple_result = ast.literal_eval(str_tuple)
        return list(tuple_result)
    else:
        return []

for i in range(len(regions)):
    directory_path = r'Path to Directory where Monte Carlo Simulations will be saved\\'+regions[i]+'/ML/'
    dfs = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    ddf = dd.concat(dfs, ignore_index=True)

    # apply your function
    meta = pd.Series(dtype=object)  # adjust this to match the output type of your function
    ddf['Gauge_Combination'] = ddf['Gauge_Combination'].map(convert_string_to_list, meta=meta)

    # select your desired columns
    ddf = ddf[['Gauge_Combination','Cummulative_Volume']]

    # compute the result
    df_result = ddf.compute().reset_index(drop = True)
    df_result['Names_length'] = df_result['Gauge_Combination'].apply(len)
    Benchmark_Vol = df_result.groupby('Names_length')['Cummulative_Volume'].mean()[len(df_result.groupby('Names_length'))]
    df_result['Error'] = (abs(df_result['Cummulative_Volume'] - Benchmark_Vol))*100/Benchmark_Vol
    min_ensemble = df_result.groupby('Names_length')['Error'].min()
    max_ensemble = df_result.groupby('Names_length')['Error'].max()
    mean_error = df_result.groupby('Names_length')['Error'].mean()
    quant95 = df_result.groupby('Names_length')['Error'].quantile(0.95)
    quant05 = df_result.groupby('Names_length')['Error'].quantile(0.05)
    # Smoothening
    if mean_error[1]<mean_error[2]:
        mean_error[1] = 3*mean_error[1]/2 + mean_error[2]
    std_error = df_result.groupby('Names_length')['Error'].std()
    if quant05[1]<quant05[2]:
        quant05[1] = 3*quant05[1]/2 + quant05[2]
    if quant95[1]<quant95[2]:
        quant95[1] = 3*quant95[1]/2 + quant95[2]

    fig,ax = plt.subplots(figsize = (8,6))
    
    marker_style = MarkerStyle('s')  # 's' for square
    marker_style._transform = marker_style.get_transform().scale(0.5, 1.4)  # scale x and y
    ax.fill_between(mean_error.index, quant05, quant95, label='90% Quantile Range',color = 'grey',alpha = 0.1,edgecolor='black')
#     if i =< 1:
    ax.plot(mean_error.index, mean_error, '--', label='Mean Error',color = 'black',marker=marker_style,mfc='black')
#     ax.fill_between(mean_error.index,min_ensemble ,max_ensemble, label='One Standard Deviation',color = 'grey',alpha = 0.1,edgecolor='black')
    ax.set_xlabel('Number of Gauge Combinations',fontsize = 19,weight='bold')
    ax.set_ylabel('Percentage Error',fontsize = 19,weight='bold')
    ax.legend()
    ax.set_title('Monte Carlo Realizations Vs Gauge Networks - '+regions[i],fontsize = 21,weight='bold')
    ax.tick_params(axis='both', which='both', length=7,labelsize = 16)
    # Step 7: Show the plot
    ax.grid(ls = '--')
#     plt.show()
